In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector

# Question 8


### Simulate data

In [2]:
X = np.random.normal(loc=35, scale=3, size=100)
noise = np.random.normal(size=100)

B_0 = 22
B_1 = -12
B_2 = 11
B_3 = 3

# Create feuture and response
Y = B_0 + B_1 * X + B_2 * X**2 + B_3 * X**3 + noise
X = pd.DataFrame({f"X^{i}": X**i for i in range(1, 11)})

# Perform forward stepwise selection
sfs = SequentialFeatureSelector(
    linear_model.LogisticRegression(),
    k_features=3,
    forward=True,
    scoring="accuracy",
    cv=None,
)
selected_features = sfs.fit(X, y)

### Perform forward stepwise selection

The selection strategy suggested $X^2$ and $X^3$ as the best predictors.

In [7]:
# Define a cp function
def nCp(sigma2, estimator, X, Y):
    """Negative Cp statistic."""
    n, p = X.shape
    Yhat = estimator.predict(X)
    RSS = np.sum((Y - Yhat) ** 2)
    return -(RSS + 2 * p * sigma2) / n

# Feutures to choose from 



('X^2', 'X^3')

### Backward Selection 

Appears to not be fully working

In [4]:
strategy_backward = Stepwise.first_peak(spec, direction='backward', max_terms=len(spec.terms))
Cp_scores_backward = sklearn_selected(OLS, strategy_backward, scoring=neg_Cp)
Cp_scores_backward.fit(df, Y)
Cp_scores_backward.selected_state_

()

### Lasso

In [6]:
# Define the model


29.40383569317111